# 🚀 ClearML Agent for Google Colab

이 노트북은 Google Colab을 ClearML Agent로 설정하여 원격 학습 환경으로 사용합니다.

## 사전 준비
1. ClearML Server가 실행 중이어야 합니다
2. ngrok 터널링이 활성화되어 있어야 합니다
3. Colab에서 GPU 런타임을 선택하세요 (런타임 → 런타임 유형 변경 → GPU)

## 워크플로우
1. 패키지 설치
2. ClearML 인증 설정
3. Agent 데몬 실행
4. 큐에서 작업 가져와 실행

## 1️⃣ 패키지 설치

In [ ]:
# ClearML 및 Agent 설치
!pip install -q clearml clearml-agent

# GPU 확인
!nvidia-smi

## 2️⃣ ClearML 인증 설정

⚠️ **아래 값들을 실제 서버 정보로 변경하세요!**

In [ ]:
# ===========================================
# 🔧 설정 값 (여기만 수정하면 됩니다)
# ===========================================

# ngrok에서 발급받은 URL들
CLEARML_WEB_HOST = "https://YOUR_NGROK_WEB.ngrok-free.app"     # Web UI (8080)
CLEARML_API_HOST = "https://YOUR_NGROK_API.ngrok-free.app"     # API Server (8008)
CLEARML_FILES_HOST = "https://YOUR_NGROK_FILES.ngrok-free.app" # File Server (8081)

# Agent 인증 키 (.env 파일의 값과 동일해야 함)
CLEARML_ACCESS_KEY = "CLEARML_AGENT_ACCESS_KEY_CHANGE_ME"
CLEARML_SECRET_KEY = "CLEARML_AGENT_SECRET_KEY_CHANGE_ME"

# 구독할 큐 목록 (쉼표로 구분)
QUEUES = "default,vision,nlp"

In [ ]:
# ClearML 인증 설정
from clearml import Task

Task.set_credentials(
    api_host=CLEARML_API_HOST,
    web_host=CLEARML_WEB_HOST,
    files_host=CLEARML_FILES_HOST,
    key=CLEARML_ACCESS_KEY,
    secret=CLEARML_SECRET_KEY
)

print("✅ ClearML 인증 설정 완료!")

## 3️⃣ 연결 테스트

In [ ]:
# 서버 연결 테스트
import requests

try:
    response = requests.get(f"{CLEARML_API_HOST}/debug.ping", timeout=10)
    if response.status_code == 200:
        print("✅ ClearML Server 연결 성공!")
        print(f"   Web UI: {CLEARML_WEB_HOST}")
    else:
        print(f"❌ 연결 실패: HTTP {response.status_code}")
except Exception as e:
    print(f"❌ 연결 오류: {e}")
    print("\n💡 확인사항:")
    print("   1. ClearML Server가 실행 중인지 확인")
    print("   2. ngrok 터널이 활성화되어 있는지 확인")
    print("   3. URL이 올바른지 확인")

## 4️⃣ ClearML Agent 설정 파일 생성

In [ ]:
# clearml.conf 파일 생성
import os

config_content = f"""
api {{
    web_server: {CLEARML_WEB_HOST}
    api_server: {CLEARML_API_HOST}
    files_server: {CLEARML_FILES_HOST}
    credentials {{
        access_key: "{CLEARML_ACCESS_KEY}"
        secret_key: "{CLEARML_SECRET_KEY}"
    }}
}}

agent {{
    worker_id: "colab-gpu-worker"
    worker_name: "Google Colab Agent"
    
    # GPU 설정
    cuda_version: auto
    cudnn_version: auto
    
    # 패키지 관리
    package_manager {{
        type: pip
        force_upgrade: true
    }}
    
    # Git 설정
    git_user: ""
    git_pass: ""
}}
"""

os.makedirs(os.path.expanduser("~/.clearml"), exist_ok=True)
with open(os.path.expanduser("~/.clearml/clearml.conf"), "w") as f:
    f.write(config_content)

print("✅ clearml.conf 파일 생성 완료!")

## 5️⃣ Agent 데몬 실행

아래 셀을 실행하면 Agent가 큐에서 작업을 기다립니다.

**⚠️ 주의**: 이 셀은 계속 실행되며, 작업이 들어올 때까지 대기합니다.

In [ ]:
# Agent 실행 (foreground 모드로 로그 확인 가능)
!clearml-agent daemon --queue {QUEUES} --foreground

---

## 📌 추가 명령어

### Agent 상태 확인

In [ ]:
# 현재 실행 중인 Agent 확인
!clearml-agent list

### 특정 Task 직접 실행

In [ ]:
# Task ID로 직접 실행
# TASK_ID = "your-task-id-here"
# !clearml-agent execute --id {TASK_ID}

### GPU 상태 모니터링

In [ ]:
# GPU 사용량 확인 (별도 셀에서 실행)
!nvidia-smi dmon -s u -d 5 -c 5